# Lectura de la vdb desde Local

In [1]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores.qdrant import Qdrant
from ml_service.embeddings import Embeddings
from utils.config import QVDB_BASE_PATH

collection_name = "sbert_embeddings"
vdb_name="all_files"
    

emb = Embeddings()
embeddings = emb.obtain_embeddings(collection_name)

client = QdrantClient(path=QVDB_BASE_PATH.format(filename=vdb_name))
qdrant = Qdrant(client, collection_name, embeddings)

# Lectura de la vdb desde Docker

In [1]:
import os

from utils.config import QDRANT_LOCAL_PATH


os.listdir(QDRANT_LOCAL_PATH)

['all_files_sbert_embeddings',
 'POL320150503_sbert_embeddings',
 'POL320180100_sbert_embeddings',
 'POL320190074_sbert_embeddings',
 'POL320200214_sbert_embeddings',
 'POL320210210_sbert_embeddings']

In [2]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores.qdrant import Qdrant
from ml_service.embeddings import Embeddings


embedding_name = "sbert_embeddings"
collection_name = "all_files_sbert_embeddings"

emb = Embeddings()
embeddings = emb.obtain_embeddings(embedding_name)

client = QdrantClient(host="localhost", port=6333)
qdrant = Qdrant(client, collection_name, embeddings)

In [10]:
retrieve = qdrant.as_retriever(search_type="mmr")

In [18]:
res = retrieve.invoke("Respecto a las obligaciones del asegurado")

print([r.metadata["file"] for r in res][:2])

best_match = ['POL320190074', 'POL320150503']

['POL320190074', 'POL320150503']


In [37]:
from pprint import pprint

pprint(res[0].page_content)

('seguro para prestaciones médicas de alto costo reglas aplicables al contrato '
 'cobertura definiciones. exclusiones. carencia obligaciones del asegurado '
 'declaraciones del asegurado primas y efectos del no pago de las primas '
 'beneficiarios denuncia de siniestros vigencia y terminación comunicación '
 'entre las partes solución de controversias clausulas adicionales domicilio '
 'cláusulas aplicables reglas aplicables al contrato obligaciones del '
 'asegurado declaraciones del asegurado cobertura y materia asegurada '
 'definiciones exclusiones denuncia de siniestros terminación')


In [19]:
retriever_list = []

for file in best_match:
    retriever_list.append(Qdrant(client, f"{file}_{embedding_name}", embeddings))

In [20]:
retriever_1 = retriever_list[0].as_retriever(search_type="mmr")
retriever_2 = retriever_list[1].as_retriever(search_type="mmr")

In [23]:
res_1 = retriever_1.invoke("Respecto a las obligaciones del asegurado")[:2]
res_1

[Document(page_content='Cualquier dificultad que se suscite entre el asegurado, el contratante o el beneficiario, según corresponda, yel asegurador, sea en relación con la validez o ineficacia del contrato de seguro, o con motivo de la interpretación o aplicación de sus condiciones generales o particulares, su cumplimiento o incumplimiento, o sobre la procedencia o el monto de una indemnización reclamada al amparo del mismo, será resuelta en los términos establecidos en el Artículo 543 del Código de Comercio. Todo lo anterior se entiende sin perjuicio del derecho que tiene el Asegurado de recurrir siempre ante el tribunal competente en ejercicio de sus derechos de consumidor conforme a la Ley 19.496. No obstante lo estipulado precedentemente, el asegurado, el contratante o beneficiario, según corresponda, podrá, por sí solo y en cualquier momento, someter al arbitraje de la Comisión del Mercado Financiero las dificultades que se susciten con la compañía cuando el monto de los daños rec

In [24]:
res_2 = retriever_2.invoke("Respecto a las obligaciones del asegurado")[:2]
res_2

[Document(page_content='El Asegurado está obligado a: 1º Declarar sinceramente todas las circunstancias que solicite el Asegurador para identificar la cosa asegurada y apreciar la extensión de los riesgos; 2° Informar, a requerimiento del Asegurador, sobre la existencia de otros seguros que amparen el mismo objeto; 3° Pagar la prima en la forma y época pactadas; 4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de cualquier hecho que pueda constituir o constituya un siniestro, y 5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y consecuencias. Si el Tomador o Contratante del seguro y el Asegurado son personas distintas, corresponde al Tomador o Contratante el cumplimiento de las obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidas por el Asegurado. Las obligaciones del Tomador o Contratante podrán ser cumplidas por el Asegurado.', metadata={'articl

In [35]:
res_1[0].metadata

{'article': 'ARTÍCULO 13º: SOLUCIÓN DE CONTROVERSIAS',
 'file': 'POL320190074',
 '_id': '50532c99-3262-474b-b521-f9772537270a',
 '_collection_name': 'POL320190074_sbert_embeddings'}

In [32]:
res_2[1].metadata


{'article': 'ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO',
 'file': 'POL320150503',
 '_id': '01488344-34d3-432a-a604-f419fc4bb706',
 '_collection_name': 'POL320150503_sbert_embeddings'}

In [34]:
from pprint import pprint

pprint(res_1[0].page_content)

('Cualquier dificultad que se suscite entre el asegurado, el contratante o el '
 'beneficiario, según corresponda, yel asegurador, sea en relación con la '
 'validez o ineficacia del contrato de seguro, o con motivo de la '
 'interpretación o aplicación de sus condiciones generales o particulares, su '
 'cumplimiento o incumplimiento, o sobre la procedencia o el monto de una '
 'indemnización reclamada al amparo del mismo, será resuelta en los términos '
 'establecidos en el Artículo 543 del Código de Comercio. Todo lo anterior se '
 'entiende sin perjuicio del derecho que tiene el Asegurado de recurrir '
 'siempre ante el tribunal competente en ejercicio de sus derechos de '
 'consumidor conforme a la Ley 19.496. No obstante lo estipulado '
 'precedentemente, el asegurado, el contratante o beneficiario, según '
 'corresponda, podrá, por sí solo y en cualquier momento, someter al arbitraje '
 'de la Comisión del Mercado Financiero las dificultades que se susciten con '
 'la compañía cu

In [8]:
retrieve2 = qdrant.as_retriever()

In [9]:
retrieve2.invoke("Respecto a las obligaciones del asegurado")

[Document(page_content='SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO REGLAS APLICABLES AL CONTRATO COBERTURA DEFINICIONES. EXCLUSIONES. CARENCIA OBLIGACIONES DEL ASEGURADO DECLARACIONES DEL ASEGURADO PRIMAS Y EFECTOS DEL NO PAGO DE LAS PRIMAS BENEFICIARIOS DENUNCIA DE SINIESTROS VIGENCIA Y TERMINACIÓN COMUNICACIÓN ENTRE LAS PARTES SOLUCIÓN DE CONTROVERSIAS CLAUSULAS ADICIONALES DOMICILIO CLÁUSULAS APLICABLES REGLAS APLICABLES AL CONTRATO OBLIGACIONES DEL ASEGURADO DECLARACIONES DEL ASEGURADO COBERTURA Y MATERIA ASEGURADA DEFINICIONES EXCLUSIONES DENUNCIA DE SINIESTROS TERMINACIÓN', metadata={'file': 'POL320190074', 'policy_header': 'SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO', '_id': '5bcab07b-d02d-450d-a253-63cf959b79db', '_collection_name': 'all_files_sbert_embeddings'}),
 Document(page_content='PÓLIZA DE SEGURO PARA PRESTACIONES MÉDICAS DERIVADAS DE ACCIDENTE REGLAS APLICABLES AL CONTRATO COBERTURA Y MATERIA ASEGURADA LIMITACIONES DE LAS COBERTURAS DEFINICIONES EXCLUSIONES OBL

## Pruebas simples modelo Cohere

In [1]:
from ml_service.retrievers import obtain_retrievers
from ml_service.simple_rag import obtain_rag_chain

In [2]:
collection_name = "sbert_embeddings"
vdb_name="all_files"

In [3]:
retriever = obtain_retrievers(vdb_name, collection_name)

In [4]:
retriever.invoke("Respiratorio")

[Document(page_content='funciones vitales del cuerpo, que temporal o permanentemente no pueden realizarse de forma\nindependiente, e incluye la respiración asistida, la nutrición e hidratación artificiales y la reanimación\ncardiopulmonar. Cuando el paciente tiene una condición tratable, el tratamiento para el mantenimiento\nartificial de la vida (incluyendo intubación y ventilación mecánica, alimentación parenteral o por gastrostomía,\ndiálisis o cualquier otro procedimiento relacionado) se usa temporalmente hasta que su condición pueda\nestabilizarse y el cuerpo pueda recuperar su función normal. Algunas veces el cuerpo no recupera, pese a\nlos tratamientos proporcionados, las funciones tratadas. En esos casos, es la condición subyacente, no el\nacto de descontinuar el tratamiento, lo que podría producir la muerte.\n \n \nMÉDICAMENTE NECESARIO: El tratamiento, servicio o suministro médico que es determinado por el\nAsegurador como necesario y apropiado para el diagnóstico y/o tratami

In [5]:
chain = obtain_rag_chain(retriever)

In [6]:
res = chain.invoke("Consideraciones para causa de insómnio")
res

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


{'context': [Document(page_content='b) Embarazo, parto normal, parto por cesárea y gastos del recién nacido.\n \nc) Enfermedad psicológica y/o psiquiátrica.\n \nd) Cirugías y/o tratamientos estéticos, cosméticos, plásticos, reparadores, maxilofaciales, ortopédicos y otros\ntratamientos para corregir malformaciones congénitas o que sean con fines de embellecimiento, a menos\nque sean necesarios para el tratamiento a causa de un accidente o complicación de intervención quirúrgica\ncausada por una patología, que ocurra mientras el Asegurado se encuentre amparado por esta Póliza.\n \ne) Lesión o enfermedad causada por guerra civil o internacional, sea que ésta haya sido declarada o no,\ninvasión y actividades u hostilidades de enemigos extranjeros.\n \nf) Lesión o enfermedad causada por participación activa del Asegurado en riñas, alborotos, rebelión,\nrevolución, motín, poder militar, sabotaje, cualquier acto contra el orden público, en acto terrorista o en actos\ncalificados por la ley c

## Test de reducción de la respuesta

In [15]:
import os
from dotenv import load_dotenv

from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.cohere import Cohere

load_dotenv()

prompt_template = """\
Debes resumir la siguiente ENTRADA a solo una expresion "Yes" o "No".
Si la ENTRADA se refiere a continuar la búsqueda es "Yes" si se refiere a detenerse "No".

ENTRADA:
{input}

EXPRESION:
"""



prompt = PromptTemplate(
    input_variables=["input"],
    template=prompt_template,
)

llm = Cohere(
    model="command",
    cohere_api_key=os.getenv("COHERE_API_KEY"),
    temperature=0
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

TypeError: must be str, not list

In [12]:
llm_chain.invoke(input="Me parece bien")

{'input': 'Me parece bien', 'text': ' Yes'}

In [13]:
llm_chain.invoke(input="Sigamos adelante")

{'input': 'Sigamos adelante', 'text': ' Yes'}

In [16]:
llm_chain.invoke(input="Hagamos la busqueda")

{'input': 'Hagamos la busqueda', 'text': ' Yes'}

In [17]:
llm_chain.invoke(input="Te hago otra pregunta")

{'input': 'Te hago otra pregunta', 'text': ' No'}

In [18]:
llm_chain.invoke(input="Contestame otra cosa")

{'input': 'Contestame otra cosa', 'text': ' No'}

# Prueba de identificación de urls

In [19]:
import os
from dotenv import load_dotenv

from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.cohere import Cohere

load_dotenv()

prompt_template = """\
Dada la siguiente ENTRADA, si encuentras una URL en el texto, devolver la URL, si hay más de una URL,\
devolver "Proceso una URL a la vez, por favor ingresar una única URL".
Si no encuentras ninguna URL, devolver "No".

ENTRADA:
{input}

SALIDA:
"""



prompt = PromptTemplate(
    input_variables=["input"],
    template=prompt_template,
)

llm = Cohere(
    model="command",
    cohere_api_key=os.getenv("COHERE_API_KEY"),
    temperature=0
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [20]:
llm_chain.invoke(
    input="Te paso la URL: https://python.langchain.com/docs/integrations/retrievers/self_query/qdrant_self_query"
)

{'input': 'Te paso la URL: https://python.langchain.com/docs/integrations/retrievers/self_query/qdrant_self_query',
 'text': ' https://python.langchain.com/docs/integrations/retrievers/self_query/qdrant_self_query'}